In [1]:
import pandas as pd
import requests
import numpy as np
from tqdm import tqdm
import time
from requests.adapters import HTTPAdapter, Retry
from functools import lru_cache

SESSION = requests.Session()
retries = Retry(total=3, backoff_factor=0.2, status_forcelist=(429, 500, 502, 503, 504))
SESSION.mount("http://", HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))
BASE = "http://localhost:5000"

In [2]:
georgia_addresses = pd.read_csv('data/georgia_addresses.csv')
dim_asset = pd.read_csv('data/DimAsset.csv', usecols=['AssetName', 'StateCode', 'Latitude', 'Longitude'])
georgia_assets = dim_asset[dim_asset['StateCode'] == 'GA'].dropna(subset=['Latitude', 'Longitude'])

In [3]:
georgia_addresses.dropna(subset=['applicant_lat', 'applicant_lon', 'employer_lat', 'employer_lon', 'asset_lat', 'asset_lon'], inplace=True)

In [4]:
@lru_cache(maxsize=100000)
def get_drive_time(lat1, lon1, lat2, lon2):
    time.sleep(0.005)
    url = f"http://localhost:5000/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    params = {"overview": "false"}  # minimal response

    try:
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()
        if not data.get("routes"):
            return None
        return data["routes"][0]["duration"] / 60  # minutes
    except Exception as e:
        print(e)
        return None

In [5]:
for i, row in tqdm(georgia_addresses.iterrows(),
                   total=len(georgia_addresses),
                   desc="Geocoding"):

    # Old vs new commute using the row's coordinates
    old_commute_time = get_drive_time(
        row['applicant_lat'], row['applicant_lon'],
        row['employer_lat'], row['employer_lon']
    )
    new_commute_time = get_drive_time(
        row['asset_lat'], row['asset_lon'],
        row['employer_lat'], row['employer_lon']
    )

    georgia_addresses.loc[i, 'old_commute'] = old_commute_time
    georgia_addresses.loc[i, 'new_commute'] = new_commute_time

    # Find closest asset (min commute time) among georgia_assets
    best_time = 1000
    best_asset = None

    for _, asset in georgia_assets.iterrows():
        commute_time = get_drive_time(
            row['employer_lat'], row['employer_lon'],
            asset['Latitude'], asset['Longitude']
        )
        if commute_time is None:
            continue
        if commute_time < best_time:
            best_time = commute_time
            best_asset = asset['AssetName']

    georgia_addresses.loc[i, 'closest_asset'] = best_asset
    georgia_addresses.loc[i, 'shortest_commute'] = best_time


Geocoding: 100%|██████████| 4553/4553 [31:38<00:00,  2.40it/s] 


In [6]:
commutes = georgia_addresses[['osl_property_id', 'asset_name', 'employer_name', 'old_commute', 'new_commute', 'closest_asset']].dropna().drop_duplicates()
commutes['is_closest_asset'] = commutes['asset_name'] == commutes['closest_asset']

commutes = commutes[(commutes['old_commute'] < 90) & (commutes['new_commute'] < 90)]

In [7]:
commutes.to_csv('data/commutes.csv')

In [8]:
print(sum(commutes['new_commute'] < commutes['old_commute']) / len(commutes))

0.5733024691358025


In [12]:
# Define bins and labels
bins = [0, 10, 20, 30, 45, float('inf')]
labels = ['<10', '10-20', '20-30', '30-45', '45+']

# Bin the new_commute values
commutes['new_commute_bin'] = pd.cut(commutes['new_commute'], bins=bins, labels=labels, right=False)

# Calculate percentage of each bin per group
bin_pct = (
    commutes.groupby(['osl_property_id', 'asset_name', 'new_commute_bin'])
    .size()
    .unstack(fill_value=0)
)

bin_pct = bin_pct.div(bin_pct.sum(axis=1), axis=0)
bin_pct = bin_pct.reset_index()

# Aggregate original commute data
commutes_agg = commutes.groupby(['osl_property_id', 'asset_name']).agg(
    avg_old_commute=('old_commute', 'mean'),
    avg_new_commute=('new_commute', 'mean'),
    closest_asset_pct=('is_closest_asset', 'mean'),
    count=('old_commute', 'count')
).reset_index()

# Merge bin percentages into the aggregated DataFrame
commutes_agg = commutes_agg.merge(bin_pct, on=['osl_property_id', 'asset_name'], how='left')

C:\Users\tyson.king\AppData\Local\Temp\ipykernel_14284\3410282942.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  commutes.groupby(['osl_property_id', 'asset_name', 'new_commute_bin'])


In [13]:
commutes_agg.to_csv('data/commutes_agg.csv')

In [11]:
display(commutes.sample(10))

,osl_property_id,asset_name,employer_name,old_commute,new_commute,closest_asset,is_closest_asset,new_commute_bin
626,2691034,Cortland Park 83,Opulent Way,57.165000,12.858333,Cortland Park 83,True,10-20
3551,3362715,Cortland at the Village,Ernst & Young,41.325000,20.048333,Cortland Old Fourth Ward,False,20-30
1248,2691034,Cortland Park 83,Saia,22.776667,22.413333,Cortland Johns Creek,False,20-30
5399,4601568,Cortland Duluth,Fivestar nail and spa,7.801667,12.603333,Somerset,False,10-20
2006,3362715,Cortland at the Village,Rays in the City,21.341667,21.026667,Cortland Old Fourth Ward,False,20-30
106,2911004,Cortland Watermark,Lockheed Martin,36.586667,18.675000,Battery 2nd Base,False,10-20
3128,3362715,Cortland at the Village,Skyex moving and storage,55.446667,10.018333,Cortland Woodlands,False,10-20
7132,4971384,Tramore Village,Home Depot,15.281667,7.748333,Tramore Village,True,<10
8302,4770692,Cortland Oleander,Berkeley Research Group,63.621667,12.036667,Cortland Peachtree Battle,False,10-20
8086,4601568,Cortland Duluth,Ingrams Wrecker Service,38.110000,54.423333,Cortland East Cobb,False,45+
